# Proyecto EDO Equipo 15

## Integrantes

- Alejandro Camacho Pérez       C-212
- Carlos Arturo Pérez Cabrera   C-212
- Diana Laura Pérez Trujillo    C-212
- David Sánchez Iglesias        C-212

## Ejercicios

Todos los ejercicios son utilizando el libro de Edwards

### Métodos y variables globales

In [6]:
from matplotlib import pyplot as pl
import enum
import pandas as pd
import prettytable as pt


EPSILON = 1e-8
SEPARATOR = "----------------------------------------------------------------------------"

class Method(enum.Enum):
    Euler = 1
    Euler_Improved = 2
    Runge_Kutta = 3


def Great_Than(a, b):
    return (not Equal_To(a, b)) and (a - b > 0)


def Less_Than(a, b):
    return (not Equal_To(a, b)) and (a - b < 0)


def Equal_To(a, b, epsilon=EPSILON):
    return abs(a - b) < epsilon


def Euler_Method(function, x, y, max, h, d):

    coordinates = []

    first_iteration = True
    while(Less_Than(x, max) or Equal_To(x, max)):

        # Save values
        if(not first_iteration):
            first_iteration = False
        else:
            coordinates.append((x, round(y, d)))

        # Update y value
        y = y+h*function(x, y)

        # Update x value
        x = x+h

    return coordinates


def Euler_Method_Improved(function, x, y, max, h, d):

    coordinates = []

    first_iteration = True
    while(Less_Than(x, max) or Equal_To(x, max)):

        # Save values
        if(first_iteration):
            first_iteration = False
        else:
            coordinates.append((x, round(y, d)))

        # Calculate first slope
        k1 = function(x, y)

        u = y+h*k1

        # Update x value
        x = x+h

        # Calculate second slope
        k2 = function(x, u)

        # Update y value
        y = y+h*(1/2)*(k1+k2)

    return coordinates


def Runge_Kutta_Method(function, x, y, max, h, d):

    coordinates = []

    first_iteration = True
    while((Less_Than(x, max) or Equal_To(x, max))):

        # Save values
        if(not first_iteration):
            first_iteration = False
        else:
            coordinates.append((x, round(y, d)))

        # Calculate first slope
        k1 = function(x, y)

        u = y+h*(1/2)*k1

        # Calculate second slope
        k2 = function(x+h/2, u)

        u = y+h*(1/2)*k2

        # Calculate third slope
        k3 = function(x+h/2, u)

        u = y+h*k3

        # Calculate fourth slope
        k4 = function(x+h, u)

        # Update y value
        y = y+h*(1/6)*(k1+2*k2+2*k3+k4)

        # Update x value
        x = x+h

    return coordinates


def Calculate_Values(funct, method: Method, min, max, y, h_values, d):

    # Calculate values
    coordinates_list = []
    for h in h_values:
        if(method == Method.Euler):
            coordinates_list.append(Euler_Method(funct, min, y, max, h, d))
        elif(method == Method.Euler_Improved):
            coordinates_list.append(Euler_Method_Improved(
                function, min, y, max, h, d))
        else:
            coordinates_list.append(
                Runge_Kutta_Method(funct, min, y, max, h, d))

    return coordinates_list


"""
    This method print a table with the coordinates values for any h value

    Parameters
    ----------
    coordinates_list : list
        List of the resulting coordinates. Each element of the list is a tuple, where the first element is a list of x values and the second element is a list of y values. The list is ordered by the h value used to calculate the coordinates.

    h_values : list
        The h values used to calculate the coordinates.

    method : Method
        The method used to calculate the coordinates.

    Returns
    -------
    None
"""


def PrintTable(coordinates_list, h_values, method: Method, n, min, max, d, y):

    # Select table name
    table_name = method.name == "Euler" and "Euler" or method.name == "Euler_Improved" and "Euler Mejorado" or "Runge-Kutta"

    # Selecting x values to print
    step_size = (max-min)/n
    x_values = [round(min + i*step_size, d) for i in range(n+1)]

    # Filling table
    epsilon = 0.01
    h_current = 0
    table = {"x":x_values}
    for coordinates_current in coordinates_list:

        # Initialize column with the name
        column_name = 'y(h={})'.format(h_values[h_current])
        table[column_name]= [y]

        x_index = 1

        # Fill column
        best_aproximation = (float('-inf'), 0)
        for x_y in coordinates_current:

            # Chek if the current value is the best aproximation to x vale to print
            if Equal_To(x_y[0], x_values[x_index], epsilon) and (Equal_To(x_y[0], best_aproximation[0]) or Great_Than(x_y[0], best_aproximation[0])):
                best_aproximation = x_y
            elif best_aproximation[0] != float('-inf'):
                table[column_name].append(best_aproximation[1])
                best_aproximation = (float('-inf'), 0)
                x_index += 1
            if(x_index == len(x_values)-1):
                table[column_name].append(coordinates_current[-1][1])
                break
        h_current += 1

    # Print table    
    pretty_table = pt.PrettyTable()
    pretty_table.title = table_name
    for key in table:
        pretty_table.add_column(key,table[key])
    print(pretty_table)



def PrintDeerInfo(data, months, limit_poblation):
    poblation = data[0][months-1][1]
    years = months/12
    print("Población de ciervos en {} años: {}".format(years, poblation))
    print("Porcentaje de población de ciervos en {} años con respecto a {}: {}%".format(years, limit_poblation, 
        round(poblation/limit_poblation*100, 2)))

    print(SEPARATOR)


### Ejercicio 24, página 132

Para el problema se requiere una computadora con
impresora. En este problema de valor inicial utilice el método de Euler mejorado con tamaños de paso h = 0.1, 0.02,
0.004 y 0.0008 para aproximar con 5 cifras decimales el valor
de la solución en 10 puntos igualmente espaciados del intervalo dado. Imprima los resultados en forma tabular con los encabezados apropiados para facilitar la comparación del efecto
de variar el tamaño de paso h. Las primas representan derivadas con respecto a x.

 - $y'= \frac{x}{1+y²},y(-1)=1;-1 \leq x \leq 1$

#### Código

In [7]:
# region Defining variables

# Function
def function(x, y):
    return x/(1+y**2)


# Initial values
x = -1
y = 1

# Interval
max = 1

# Decimal
d = 5

# Steps
h_values = [0.1, 0.02, 0.004, 0.0008]
n = 10

# endregion

# Aply method
coordinates_list = Calculate_Values(
    function, Method.Euler_Improved, x, max, y, h_values, d)


# Print
PrintTable(coordinates_list, h_values, Method.Euler_Improved, n, x, max,d,y)


+--------------------------------------------------------+
|                     Euler Mejorado                     |
+------+----------+-----------+------------+-------------+
|  x   | y(h=0.1) | y(h=0.02) | y(h=0.004) | y(h=0.0008) |
+------+----------+-----------+------------+-------------+
| -1.0 |    1     |     1     |     1      |      1      |
| -0.8 | 0.90572  |  0.90569  |  0.90219   |   0.90149   |
| -0.6 | 0.82574  |  0.82569  |  0.82285   |   0.82229   |
| -0.4 | 0.76449  |  0.76443  |  0.76243   |   0.76204   |
| -0.2 | 0.72593  |  0.72586  |  0.72483   |   0.72463   |
| 0.0  | 0.71276  |  0.71268  |   0.7127   |   0.71271   |
| 0.2  | 0.72595  |  0.72586  |  0.72692   |   0.72714   |
| 0.4  | 0.76453  |  0.76443  |  0.76647   |   0.76688   |
| 0.6  | 0.82579  |  0.82569  |  0.82856   |   0.82914   |
| 0.8  | 0.90578  |  0.90569  |  0.90922   |   0.90993   |
| 1.0  | 1.00006  |    1.0    |    1.0     |     1.0     |
+------+----------+-----------+------------+------------

#### Resultados

$$
\begin{array}{|c|c|c|c|c|}
\hline
x & y(h=0.1) & y(h=0.02) & y(h=0.004) & y(h=0.0008) \\
\hline
-1.0 & 1 & 1 & 1 & 1 \\
-0.8 & 0.90572 & 0.90569 & 0.90219 & 0.90149 \\
-0.6 & 0.82574 & 0.82569 & 0.82285 & 0.82229 \\
-0.4 & 0.76449 & 0.76443 & 0.76243 & 0.76204 \\
-0.2 & 0.72593 & 0.72586 & 0.72483 & 0.72463 \\
0.0 & 0.71276 & 0.71268 & 0.7127 & 0.71271 \\
0.2 & 0.72595 & 0.72586 & 0.72692 & 0.72714 \\
0.4 & 0.76453 & 0.76443 & 0.76647 & 0.76688 \\
0.6 & 0.82579 & 0.82569 & 0.82856 & 0.82914 \\
0.8 & 0.90578 & 0.90569 & 0.90922 & 0.90993 \\
1.0 & 1.00006 & 1.0 & 1.0 & 1.0 \\
\hline
\end{array}
$$

### Ejercicio 24, Página 142

Para el problema se requiere una computadora con
impresora. En estos problemas de valor inicial utilice el méto-
do de Runge-Kutta con tamaños de paso h = 0.2 , 01, 0.05 y
0.025 para aproximar a 6 cifras decimales los valores de la
solución en 5 puntos igualmente espaciados del intervalo
dado. Imprima los resultados en forma tabular con un enca-
bezado apropiado que facilite la comparación del efecto de
variar el tamaño de paso h. Las primas representan derivadas
con respecto a x.

 - $y'= \frac{x}{1+y²},y(-1)=1;-1 \leq x \leq 1$

#### Código

In [8]:
#region Defining variables

# Function
def function(x, y):
    return x/(1+y**2)


# Initial values
x = -1
y = 1

# Interval
max = 1

# Decimal
d = 6

# Steps
h_values = [0.2, 0.1, 0.05, 0.025]

n=5
# endregion

# Aply method
coordinates_list = Calculate_Values(
    function, Method.Runge_Kutta, x, max, y, h_values, d)


# Print

PrintTable(coordinates_list, h_values, Method.Runge_Kutta, n, x, max,d,y)


+-----------------------------------------------------+
|                     Runge-Kutta                     |
+------+----------+----------+-----------+------------+
|  x   | y(h=0.2) | y(h=0.1) | y(h=0.05) | y(h=0.025) |
+------+----------+----------+-----------+------------+
| -1.0 |    1     |    1     |     1     |     1      |
| -0.6 | 0.82569  | 0.825691 |  0.825691 |  0.825691  |
| -0.2 | 0.725856 | 0.725856 |  0.725857 |  0.725857  |
| 0.2  | 0.725856 | 0.725856 |  0.725857 |  0.725857  |
| 0.6  | 0.82569  | 0.825691 |  0.825691 |  0.825691  |
| 1.0  |   1.0    |   1.0    |    1.0    |    1.0     |
+------+----------+----------+-----------+------------+


#### Resultados

$$
\begin{array}{|c|c|c|c|c|}
\hline
x & y(h=0.2) & y(h=0.1) & y(h=0.05) & y(h=0.025) \\
\hline
-1.0 & 1 & 1 & 1 & 1 \\
-0.6 & 0.82569 & 0.825691 & 0.825691 & 0.825691 \\
-0.2 & 0.725856 & 0.725856 & 0.725857 & 0.725857 \\
0.2 & 0.725856 & 0.725856 & 0.725857 & 0.725857 \\
0.6 & 0.82569 & 0.825691 & 0.825691 & 0.825691 \\
1.0 & 1 & 1 & 1 & 1 \\
\hline
\end{array}
$$

### Ejercicio 26, página 132

Suponga que
en un pequeño bosque la población de venados P(t) inicialmente es de 25 individuos y satisface la ecuación
logística  

- $\frac{dP}{dt} = 0.0225P − 0.0003P²$  

(con t en meses). Utilice el método de Euler con
una calculadora programable o una computadora para aproximar la solución a 10 años, primero con un tamaño de paso h = 1 y después con h = 0.5, redondeando
los valores aproximados de P a números enteros de venados. ¿Qué
porcentaje de la población límite de 75 venados se obtiene
después de 5 años? ¿Después de 10 años?


#### Código

In [9]:
# region Defining variables

# Function
def function(x, y):
    return 0.0225*y-0.0003*(y**2)


# Initial values
x = 0
y = 25

# Interval
max=120

# Decimals
d=0

# Steps
h_values=[1,0.5]
n=12

# Aply method
deer_data = Calculate_Values(
    function, Method.Euler, x, max, y, h_values, d)


# Print
PrintTable(deer_data, h_values, Method.Euler, n, x, max,d,y)

# Print info
PrintDeerInfo(deer_data, 60, 75)
PrintDeerInfo(deer_data, 120, 75)



+---------------------------+
|           Euler           |
+-------+--------+----------+
|   x   | y(h=1) | y(h=0.5) |
+-------+--------+----------+
|  0.0  |   25   |    25    |
|  10.0 |  29.0  |   29.0   |
|  20.0 |  33.0  |   33.0   |
|  30.0 |  37.0  |   37.0   |
|  40.0 |  41.0  |   41.0   |
|  50.0 |  45.0  |   45.0   |
|  60.0 |  49.0  |   49.0   |
|  70.0 |  53.0  |   53.0   |
|  80.0 |  56.0  |   56.0   |
|  90.0 |  59.0  |   59.0   |
| 100.0 |  62.0  |   62.0   |
| 110.0 |  64.0  |   64.0   |
| 120.0 |  66.0  |   66.0   |
+-------+--------+----------+
Poblacion de ciervos en 5.0 años: 49.0
Porcentaje de poblacion de ciervos en 5.0 años con respecto a 75: 65.33%
----------------------------------------------------------------------------
Poblacion de ciervos en 10.0 años: 66.0
Porcentaje de poblacion de ciervos en 10.0 años con respecto a 75: 88.0%
----------------------------------------------------------------------------


#### Resultados

$$
\begin{array}{|c|c|c|}
\hline
x & y(h=1) & y(h=0.5) \\
\hline
0.0 & 25 & 25 \\
10.0 & 29 & 29 \\
20.0 & 33 & 33 \\
30.0 & 37 & 37 \\
40.0 & 41 & 41 \\
50.0 & 45 & 45 \\
60.0 & 49 & 49 \\
70.0 & 53 & 53 \\
80.0 & 56 & 56 \\
90.0 & 59 & 59 \\
100.0 & 62 & 62 \\
110.0 & 64 & 64 \\
120.0 & 66 & 66 \\
\hline
\end{array}
$$

- Poblacion de ciervos en $5$ años: $49$
- Porcentaje de poblacion de ciervos en $5$ años con respecto a $75: 65.33%$

- Poblacion de ciervos en $10$ años: $66$
- Porcentaje de poblacion de ciervos en 10 años con respecto a $75: 88.0%$




